# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [ ]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-04-05 07:51:11--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  28.10M   161MB/s    in 0.2s    

2023-04-05 07:51:11 (161 MB/s) - ‘train.csv’ saved [29470338/29470338]



,label,title,lead,text
90289,sport,Holyfield can #39;t see the end,Evander Holyfield may have finally run out of ...,Holyfield can #39;t see the end Evander Holyfi...
57351,sci/tech,IBM Fashions New Blades (NewsFactor),NewsFactor - IBM (NYSE: IBM) is upgrading its ...,IBM Fashions New Blades (NewsFactor) NewsFacto...
74186,world,Lawmakers Back Sharon on Plan for Leaving Gaza,"The vote, in an atmosphere of high drama and t...",Lawmakers Back Sharon on Plan for Leaving Gaza...
55112,sport,Works in Progress,U-Md.'s Joel Statham and Georgia Tech's Reggie...,Works in Progress U-Md.'s Joel Statham and Geo...
93914,business,"US Economy: Consumer Prices, Industrial Produc...","US consumer prices rose in October, spurred by...","US Economy: Consumer Prices, Industrial Produc..."


## Hugginface Transformers

In [ ]:
!pip install transformers
from transformers import DistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
import torch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

cuda


In [ ]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text
model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)


In [ ]:
##TODO print the summary of the model
print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
##TODO prepare the dataset for torch.
## transform labels
cat_to_int = {cat: i for i, cat in enumerate(df['label'].unique())}
df['label_int'] = df['label'].map(cat_to_int)
df.head()

,label,title,lead,text,label_int
90289,sport,Holyfield can #39;t see the end,Evander Holyfield may have finally run out of ...,Holyfield can #39;t see the end Evander Holyfi...,0
57351,sci/tech,IBM Fashions New Blades (NewsFactor),NewsFactor - IBM (NYSE: IBM) is upgrading its ...,IBM Fashions New Blades (NewsFactor) NewsFacto...,1
74186,world,Lawmakers Back Sharon on Plan for Leaving Gaza,"The vote, in an atmosphere of high drama and t...",Lawmakers Back Sharon on Plan for Leaving Gaza...,2
55112,sport,Works in Progress,U-Md.'s Joel Statham and Georgia Tech's Reggie...,Works in Progress U-Md.'s Joel Statham and Geo...,0
93914,business,"US Economy: Consumer Prices, Industrial Produc...","US consumer prices rose in October, spurred by...","US Economy: Consumer Prices, Industrial Produc...",3


In [ ]:
##TODO prepare the dataset for torch.
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

inputs = tokenizer(df['text'].tolist(), return_tensors="pt", padding=True, truncation=True)
labels = torch.tensor(df['label_int'].tolist()).long() 
#print(inputs, labels)


tensor([0, 1, 2,  ..., 1, 3, 2])


In [ ]:
import numpy as np
unique_labels, counts = np.unique(df["label_int"], return_counts=True)
print (unique_labels, counts)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_labels))

optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},  
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

[0 1 2 3] [2456 2546 2513 2485]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [ ]:
##TODO split the sample into a training and a test set 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label_int'].tolist(), test_size=.2, stratify=df['label_int'])

# generate batches
batch_size = 8

# calculate the number of batches based on the batch size
num_train_batches = len(X_train) // batch_size
num_test_batches = len(X_test) // batch_size

# trim the data to fit the batch size
X_train = X_train[:num_train_batches * batch_size]
X_test = X_test[:num_test_batches * batch_size]
y_train = y_train[:num_train_batches * batch_size]
y_test = y_test[:num_test_batches * batch_size]

# convert the lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# reshape the data into batches
X_train = X_train.reshape(-1, batch_size)
X_test = X_test.reshape(-1, batch_size)
y_train = y_train.reshape(-1, batch_size)
y_test = y_test.reshape(-1, batch_size)

X_train, X_test = X_train.tolist(), X_test.tolist()

In [ ]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)

# train
from tqdm import tqdm

model.to(device)
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
        # prepare model input through our tokenizer
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        # place everything on the right device
        model_inputs = {k:v.to(device) for k,v in model_inputs.items()}
        # labels have to be torch long tensors
        labels = torch.tensor(labels).long().to(device)
        # now, we can perform the forward pass
        output = model(**model_inputs, labels=labels)
        loss, logits = output[:2]
        # and the backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()



100%|██████████| 1000/1000 [04:22<00:00,  3.80it/s]


In [ ]:
predictions, targets = [], []
model.eval()

with torch.no_grad():
    for text, labels in tqdm(zip(X_test, y_test), total=len(X_test)):
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        model_inputs = {k:v.to(device) for k,v in model_inputs.items()}

        output = model(**model_inputs)
        logits = output[0]
        # prediction is the argmax of the logits
        predictions.extend(logits.argmax(dim=1).tolist())
        targets.extend(labels)
        
from sklearn import metrics
accuracy = metrics.accuracy_score(targets, predictions)
print ("accuracy", accuracy)
classification_report = metrics.classification_report(targets, predictions)
print (classification_report)

100%|██████████| 250/250 [00:21<00:00, 11.88it/s]

accuracy 0.908
              precision    recall  f1-score   support

           0       0.95      0.98      0.97       491
           1       0.88      0.91      0.89       509
           2       0.90      0.91      0.91       503
           3       0.90      0.83      0.87       497

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000



# Doc Embedding

In [ ]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

In [ ]:
# load the data
def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

('A person is on a baseball team.',
 'A person is playing basketball on a team.',
 2.4)

In [ ]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [ ]:
##TODO train Doc2Vec on the texts in the dataset 
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from random import shuffle

texts = text_a + text_b

# tokenize
docs_a = []
for text in text_a:
    docs_a += [word_tokenize(text)]
docs_b = []
for text in text_b:
    docs_b += [word_tokenize(text)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Doc2Vec (a)
doc_iterator_a = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs_a)]
d2v_a = Doc2Vec(doc_iterator_a,
                min_count=10, # minimum word count
                window=10,    # window size
                vector_size=100, # size of document vector
                sample=1e-4, 
                negative=5, 
                workers=4, # threads
                max_vocab_size=1000) # max vocab size

doc_iterator_b = [TaggedDocument(doc, [i]) for i, doc in enumerate(docs_b)]
d2v_b = Doc2Vec(doc_iterator_b,
                min_count=10, # minimum word count
                window=10,    # window size
                vector_size=100, # size of document vector
                sample=1e-4, 
                negative=5, 
                workers=4, # threads
                max_vocab_size=1000) # max vocab size              

# matrix of all document vectors:
D_a = d2v_a.dv.vectors
D_b = d2v_b.dv.vectors
D_a.shape, D_b.shape 

((250, 100), (250, 100))

In [ ]:
##TODO derive the word vectors for each text in the dataset 
vectors_a = [d2v_a.infer_vector(text_a)]
vectors_b = [d2v_b.infer_vector(text_b)]

##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
similarity_matrix = cosine_similarity(vectors_a[0], vectors_b[0])
print(similarity_matrix)


## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

0.0033083113


In [ ]:
##TODO do the same with embeddings provided by spaCy

In [ ]:
##TODO do the same with SBERT embeddings